In [7]:
import numpy as np
import keras.backend as K
from utils.coco_utils import load_coco_data
from keras.models import Model
from keras.layers import Input, Dense, Embedding, LSTM, Reshape, concatenate
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.losses import SparseCategoricalCrossentropy

data = load_coco_data(pca_features=True)

In [2]:
#calculate the length of each caption in the training and validation sets
data["train_captions_lens"] = np.zeros(data["train_captions"].shape[0])
data["val_captions_lens"] = np.zeros(data["val_captions"].shape[0])
for i in range(data["train_captions"].shape[0]) :
    data["train_captions_lens"][i] = np.nonzero(data["train_captions"][i] == 2)[0][0] + 1
for i in range(data["val_captions"].shape[0]) :
    data["val_captions_lens"][i] = np.nonzero(data["val_captions"][i] == 2)[0][0] + 1

In [9]:
max_length = 0
for caption in data["train_captions"]:
    if len(caption) > max_length:
        max_length = len(caption)
for caption in data["val_captions"]:
    if len(caption) > max_length:
        max_length = len(caption)
data["max_length"] = max_length

word_set = set()
for caption in data["train_captions"]:
    for word in caption:
        word_set.add(word)
for caption in data["val_captions"]:
    for word in caption:
        word_set.add(word)
data["vocab_size"] = len(word_set) + 1 # add 1 for the <PAD> token



In [10]:
#define the R-LSTM model
input_image = Input(shape=(data["train_features"].shape[1],))
encoded_image = Dense(256, activation='relu')(input_image)

input_caption = Input(shape=(data["max_length"],))
embedded_caption = Embedding(data["vocab_size"], 256, mask_zero=True)(input_caption)
encoded_caption = LSTM(256)(embedded_caption)

merged = concatenate([encoded_image, encoded_caption])
output = Dense(data["vocab_size"], activation='softmax')(merged)

model = Model(inputs=[input_image, input_caption], outputs=output)
model.compile(loss=SparseCategoricalCrossentropy(), optimizer='adam')

In [11]:
def caption_generator(features, captions, vocab_size, batch_size):
    num_samples = features.shape[0]
    num_batches = num_samples // batch_size
    
    while True:
        for i in range(num_batches):
            batch_features = features[i*batch_size:(i+1)*batch_size]
            batch_captions = captions[i*batch_size:(i+1)*batch_size]
            batch_captions_onehot = to_categorical(batch_captions, num_classes=vocab_size)
            yield ([batch_features, batch_captions], batch_captions_onehot)

batch_size = 128
train_steps_per_epoch = data["train_features"].shape[0] // batch_size
val_steps_per_epoch = data["val_features"].shape[0] // batch_size

train_generator = caption_generator(data["train_features"], data["train_captions"], data["vocab_size"], batch_size)
val_generator = caption_generator(data["val_features"], data["val_captions"], data["vocab_size"], batch_size)


In [13]:
#train the model
model.fit(train_generator, steps_per_epoch=train_steps_per_epoch, epochs=50,
          validation_data=val_generator, validation_steps=val_steps_per_epoch)

Epoch 1/50


InvalidArgumentError: Graph execution error:

Detected at node 'sparse_categorical_crossentropy/remove_squeezable_dimensions/Squeeze' defined at (most recent call last):
    File "c:\Users\asus\AppData\Local\Programs\Python\Python37\lib\runpy.py", line 193, in _run_module_as_main
      "__main__", mod_spec)
    File "c:\Users\asus\AppData\Local\Programs\Python\Python37\lib\runpy.py", line 85, in _run_code
      exec(code, run_globals)
    File "c:\Users\asus\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "c:\Users\asus\AppData\Local\Programs\Python\Python37\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "c:\Users\asus\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel\kernelapp.py", line 583, in start
      self.io_loop.start()
    File "c:\Users\asus\AppData\Local\Programs\Python\Python37\lib\site-packages\tornado\

In [ ]:
model.save("reference_lstm_model.h5")